In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import autograd.numpy as anp
import pymanopt
import pymanopt.manifolds
import pymanopt.optimizers

from sklearn.datasets import make_blobs
import graphlearning as gl
import numpy as np

import matplotlib.pyplot as plt

2024-04-19 18:05:52.227436: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/chester/.mujoco/mjpro150/bin:/home/chester/.mujoco/mjpro150/bin
2024-04-19 18:05:52.284139: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/chester/.mujoco/mjpro150/bin:/home/chester/.mujoco/mjpro150/bin
2024-04-19 18:05:52.288058: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/chester/.mujoco/mjpro150/bin:/home/che

In [3]:
import graphlearning as gl
from graphlearning.ssl import ssl

import numpy as np
from jax import lax, vmap
from jax import numpy as jnp
from jax.config import config
from jax.scipy.sparse.linalg import cg as jcg
config.update("jax_enable_x64", True)

import matplotlib.pyplot as plt
import sklearn.datasets as datasets

import scipy
from scipy import sparse as sp
from scipy.linalg import solve as ssolve
from scipy.linalg import svd

from tqdm.notebook import tqdm

import time

In [4]:
labels = gl.datasets.load('mnist', labels_only=True)
W = gl.weightmatrix.knn('mnist', 10, metric='raw')
G = gl.graph(W)
    
train_ind = gl.trainsets.generate(labels, rate=1, seed=42)
train_labels = labels[train_ind]

W.shape

(70000, 70000)

In [5]:
def F(X, A, B, C):
  C_sqrt = _sqrtm(C)
  return  np.trace(np.dot(X.T, A(X)@C)) - 2*np.trace(np.dot(X.T, B))

def value_and_gradF(X, A, B, C):
  C_sqrt = _sqrtm(C)
  return F(X, A, B, C), A(X)@C - B

def Lk(X, A, B, C):
  C_sqrt = _sqrtm(C)
  return X.T@(A(X)@C - B)

def ssmsubspace(X_k, Z, A, B, Vg, C):
    C_sqrt = _sqrtm(C)
    AXE = A(X_k)@C - B
    S = [X_k, AXE, Vg, Z]
    Q, R = np.linalg.qr(np.concatenate(S,axis=-1), mode='reduced')
    return Q

def newton(Lambdak, Xk, A, B, Vxg, C, z0):
  C_inv = np.linalg.pinv(C)
  C_sqrt = _sqrtm(C)
  Po = lambda z : z - Vxg@(Vxg.T@z)

  rhs = Po(B + Xk@Lambdak - A(Xk)@C)
  lhs = lambda z :                                                                                                                             Po(A(Po(z))@C) - z@Lambdak
  z = conjgrad(lhs, rhs, x0=z0, max_iter=100)
    
  return z

In [6]:
def solve_svd(A,b):
    U,s,Vh = svd(A)
    c = np.dot(U.T,b)
    w = np.dot(np.diag(1/s),c)
    x = np.dot(Vh.conj().T,w)
    return x

def _sqrtm(C):
    # Computing diagonalization
    evalues, evectors = np.linalg.eig(C)
    # Ensuring square root matrix exists
    sqrt_matrix = evectors @ np.diag(np.sqrt(evalues)) @ np.linalg.inv(evectors)
    return sqrt_matrix

def eig(A, eps=1e-7):
    w, v = np.linalg.eig(A)
    w[w < eps] = 0
    sidx = np.argsort(w)
    idx = sidx[np.in1d(sidx, np.flatnonzero(w!=0))]
    return w[idx], v[:,idx]

def negdef_transform(X, B, C):
  """Computation of Q (for initialization only) """
  U_E, _, V_E = np.linalg.svd(X.T@B, full_matrices=False)
  X = X@(U_E@V_E)
  return X

def stproject(x):
  U_x, _, V_xt = jnp.linalg.svd(x, full_matrices=False)
  return U_x@V_xt

def conjgrad(A, b, x0=None, max_iter=400, tol=1e-5):
    """Conjugate Gradient Method to solve Ax = B """
    if x0 is None:
      x = np.zeros_like(b)
    else:
      x = x0

    r = b - A(x)
    p = r
    rsold = np.sum(r**2,axis=0)

    err = 1
    i = 0
    while (err > tol) and (i < max_iter):
        i += 1
        Ap = A(p)
        alpha = rsold / np.sum(p*Ap,axis=0)
        x += alpha * p
        r -= alpha * Ap
        rsnew = np.sum(r**2,axis=0)
        err = np.sqrt(np.sum(rsnew))
        p = r + (rsnew / rsold) * p
        rsold = rsnew

    return x

# Geometric progression: step sizes for line search
beta=0.9
step_sizes = 1*np.power(beta,np.linspace(0,100,num=100))

#step_sizes = np.append(step_sizes,0)
step_sizes=np.array(step_sizes)

In [7]:
def rgdstep(A_, B_, C_, Xk):
  def _line_search(s, gXk_, Xk_, A_, B_, C_):
    Xk_ = Xk_ - s*gXk_
    Xk_ = stproject(Xk_)

    _f_k = jnp.trace(jnp.dot(Xk_.T, A_@Xk_@C_)) - 2*jnp.trace(jnp.dot(Xk_.T, B_))
    return _f_k, Xk_

  Lambk = Xk.T@(A_@Xk@C_ - B_)
  Lambksym = 0.5*(Lambk + Lambk.T)
  gXk = (A_@Xk@C_ - B_) - 0.5*(Xk@Lambksym)
  _Fks, _Xks = vmap(_line_search,in_axes=(0,None,None,None,None,None))(step_sizes,gXk,Xk,A_,B_,C_)
  _f_kamin = np.argmin(_Fks)
  Xk = _Xks[_f_kamin]

  return Xk, _Fks[_f_kamin]

def rndstep(A_, B_, C_, Xk, dr=10000, si=0.0, eps=0.1):
  C_inv = np.linalg.pinv(C_).real
  C_inv_sqrt = _sqrtm(C_inv).real
  C_sqrt = _sqrtm(C_).real
  def _line_search(s, gXk_, Xk_, A_, B_, C_):
    Xk_ = Xk_ + s*gXk_
    Xk_ = stproject(Xk_)

    _f_k = jnp.trace(jnp.dot(Xk_.T, A_@Xk_@C_)) - 2*jnp.trace(jnp.dot(Xk_.T, B_))
    return _f_k, Xk_

  Lambk = Xk.T@(A_@Xk@C_ - B_)
  Lambksym = 0.5*(Lambk + Lambk.T)
  g, U = np.linalg.eig(C_inv_sqrt@Lambksym@C_inv_sqrt)
  g = np.minimum(g, dr - 0.9*si)
  Lambdak = C_sqrt@U@np.diag(g)@U.T@C_sqrt

  projx = lambda Y : Y - Xk@(0.5*(Xk.T@Y + Y.T@Xk))
  rhs = projx(-A_@Xk@C_ + B_ + Xk@Lambdak)
  lhs = lambda z : projx(A_@projx(z)@C_) - projx(z)@Lambdak

  zk = conjgrad(lhs, rhs, max_iter=10)

  _Fks, _Xks = vmap(_line_search,in_axes=(0,None,None,None,None,None))(step_sizes,zk,Xk,A_,B_,C_)
  _f_kamin = np.argmin(_Fks)
  Xk = _Xks[_f_kamin]

  return Xk, _Fks[_f_kamin], zk

def sqpstep(A_, B_, C_, Xk):
  def _line_search(s, gXk_, Xk_, A_, B_, C_):
    Xk_ = Xk_ + s*gXk_
    Xk_ = stproject(Xk_)

    _f_k = jnp.trace(jnp.dot(Xk_.T, A_@Xk_@C_)) - 2*jnp.trace(jnp.dot(Xk_.T, B_))
    return _f_k, Xk_

  Lambk = Xk.T@(A_@Xk@C_ - B_)
  Lambksym = 0.5*(Lambk + Lambk.T)
  rhs = A_@Xk@C_ - B_ - Xk@Lambksym
  projx = lambda U : U - Xk@(0.5*(Xk.T@U + U.T@Xk))
  lhs = lambda z : projx(A_@z@C_ - z@Lambksym)

  zk = conjgrad(lhs, -rhs)

  _Fks, _Xks = vmap(_line_search,in_axes=(0,None,None,None,None,None))(step_sizes,zk,Xk,A_,B_,C_)
  _f_kamin = np.argmin(_Fks)
  Xk = _Xks[_f_kamin]

  return Xk, _Fks[_f_kamin]

In [10]:
from types import LambdaType
class ssm(ssl):
    def __init__(self, W=None, class_priors=None, normalization='combinatorial', train_ind=None, 
                 labels=None, label_prior=None):
        super().__init__(W,class_priors)
        self.normalization = normalization
        self.W = W
        self.G = gl.graph(W)
        self.L = self.G.laplacian(normalization=self.normalization)
        self.d = self.G.degree_vector()
        self.M = self.L.shape[0]
        self.labels=labels
        self.train_ind = train_ind
        self.labels =labels
        self.label_prior = label_prior

    def _fit(self, train_ind, train_labels, all_labels=None):
        """sort labels by vertex id """
        idx = np.argsort(train_ind)
        train_ind = train_ind[idx]
        train_labels = train_labels[idx]

        """get masks of labeled / unlabeled vertices """
        m = train_labels.shape[0]
        n = self.M - m
        r = train_labels.max()+1
        labeled_mask = np.zeros(self.M,dtype=bool)
        labeled_mask[train_ind] = 1
        unlabeled_mask = ~labeled_mask

        """partition laplacian into labeled and unlabeled parts """
        laplacian_unlabeled_submatrix = self.L[unlabeled_mask,:][:,unlabeled_mask]
        laplacian_labeled_unlabeled_submatrix = self.L[labeled_mask,:][:,unlabeled_mask]
        laplacian_unlabeled_labeled_submatrix = self.L[unlabeled_mask,:][:,labeled_mask]

        """projection onto set of mean-zero vectors """
        o = 1/np.sqrt(n)*np.expand_dims(np.ones(n),1)
        P = lambda x : x - o@(o.T@x)

        """labeled-unlabeled affine term """
        Y = gl.utils.labels_to_onehot(train_labels)
        
        c = np.reshape(self.label_prior*self.M, (r,1))
        cl = Y.sum(0,keepdims=True).T
        rt = cu = c - cl

        C = Ct = np.diag(c.squeeze()) - Y.T@Y - 1/n * cu@cu.T
        Ct_sqrt = _sqrtm(C).real
        
        wc, Vc = np.linalg.eigh(C)
        C = np.diag(wc[1:])
        Q = Vc[:,1:]

        C_inv = np.linalg.pinv(C).real
        C_inv_sqrt = _sqrtm(C_inv).real
        C_sqrt = _sqrtm(C).real

        B0 = -P(1/n*laplacian_unlabeled_submatrix@np.ones((n,1))@rt.T + laplacian_unlabeled_labeled_submatrix@Y)
        B = B0@Q@C_sqrt
        r1 = B.shape[1]

        """PLuP """
        A = lambda x : P(laplacian_unlabeled_submatrix@P(x))

        """Eigenvectors of A """
        A_ = scipy.sparse.linalg.LinearOperator(laplacian_unlabeled_submatrix.shape, A)
        X = np.random.randn(n, r1+3)
        l, Vg = scipy.sparse.linalg.lobpcg(A_, X, M=None, tol=1e-8, largest=False,
                                          verbosityLevel=0,
                                          maxiter=1000)
        print(l)
        X = X[:,1:]
        Vg = Vg[:,1:]
        l = l[1:]

        vgterm = lambda x : np.array([(l[r1] - l[i]) * Vg[:,[i]]@(Vg[:,[i]].T@x) for i in range(r1)]).sum(0)
        Atilde0 = lambda x : A(x) + vgterm(x)
        
        Atilde0_ = scipy.sparse.linalg.LinearOperator(laplacian_unlabeled_submatrix.shape, Atilde0)
        
        dr = l[r1]
        dr1 = l[r1+1]
        drC = dr*C
        sdrC = np.linalg.eigvals(drC).min().real
        Xk = negdef_transform(Vg, B, C)
        
        def _line_searchLambda(s, Gk, Lamb, Xk, A_, B_, C):
          I = np.eye(Xk.shape[1])
          resid = (Xk.T@Xk - I)
          Lamb = Lamb + s * resid
          _f_k = F(Xk, A_, B_, C) - 0.5*jnp.trace(Lamb.T@resid)
          return _f_k, Lamb

        z = None
        Fbest = 1e10
        Xbest = None

        Fks = []
        FOCs = []
        sls = []
        Xks = []
        Bks = []
        lambks = []
        Vxgs = []
        conds = []
        params = (A, Atilde0, B, C)
        ts = []
        for k in tqdm(range(41)):
          # (i)
          t = time.time()
          Lambdak = Lk(Xk, A, B, C)
          Lambdak = Lambdak.real
          sl = np.linalg.eigvals(Lambdak).max()
          slcinv = np.linalg.eigvals(Lambdak@np.linalg.inv(C)).max()
          slsym = np.linalg.eigvals(0.5*(Lambdak + Lambdak.T)).max()
          foc = np.linalg.norm(A(Xk)@C - B - Xk@Lambdak)
          Lambdak_sym = 0.5*(Lambdak + Lambdak.T)
          Fk = F(Xk, A, B, C)
          if Fk < Fbest:
            Fbest = Fk
            Xbest = Xk
          Fks.append(Fk)
          FOCs.append(foc)
          sls.append(slcinv)
         
          print(f'{k} F: {Fk:.3f} foc: {foc:.3f} sl: {sl:.3f} slsym: {slsym:.3f} sdrC: {sdrC:.3f} slCinv: {slcinv:.3f} d_r+1 - dr: {dr1 - dr:.5f}')

          Vxg, _ = np.linalg.qr(np.concatenate([Xk, Vg],axis=-1), mode='reduced')
          Vxgs.append(Vxg)

          Bk = B + Atilde0(Xk@C) - A(Xk@C)
          Bks.append(Bk)
          z = newton(Lambdak_sym, Xk, Atilde0, Bk, Vxg, C, z0=None)

          Vk = ssmsubspace(Xk, z, A, B, Vg, C)
          Vk = Vk.real

          Atilde = Vk.T@A(Vk)
          _Atilde = lambda x : Atilde@x
          Bk = Vk.T@Bk

          U, S, Vh = np.linalg.svd(Vg.T@B@C_inv,full_matrices=False)
          si = S[-1]

          Xtilde = Vk.T@Xk

          Lambdak = Lk(Xtilde, _Atilde, Bk, C)
          Lambdak = 0.5 * (Lambdak+Lambdak.T)
          sl = np.linalg.eigvals(Lambdak).max()
          ind = 0
          foc = 1

          while ((sl >= sdrC) and (foc >= 1e-4)) or (ind < 100):
            """Riemanian newton / other algs """
            Xtilde, Fk, zk = rndstep(Atilde, Bk, C, Xtilde, dr=dr, si=si)
            Lambdak = Lk(Xtilde, _Atilde, Bk, C)
            sl = np.linalg.eigvals(Lambdak).max().real

            foc = np.linalg.norm(Atilde@Xtilde@C - Bk - Xtilde@Lambdak)
            if ind % 20 == 0:
              print(f'==== VF: {Fk:.3f} foc: {foc:.3f} sl: {sl:.3f} sdrC: {sdrC:.3f}')
            ind+=1

          Xk = Vk@Xtilde
          Xk = Xk.real
          Xks.append(Xk)
          lambks.append(Lambdak)

          u = Xk@Q.T@Ct_sqrt + 1/n*np.ones((Xk.shape[0],1))@rt.T
          pr = np.zeros((self.M,r))
          pr[labeled_mask] = gl.utils.labels_to_onehot(train_labels)
          pr[unlabeled_mask] = u
          pred_labels = pr.argmax(1)
          accuracy = gl.ssl.ssl_accuracy(pred_labels, self.labels, train_ind)
          print("accuracy: %.2f%%"%accuracy)
          ts.append(time.time() - t)

        u = Xk@Q.T@Ct_sqrt + 1/n*np.ones((Xk.shape[0],1))@rt.T

        """reshape solution """
        pr = np.zeros((self.M,r))
        pr[labeled_mask] = gl.utils.labels_to_onehot(train_labels)
        pr[unlabeled_mask] = u

        return pr, (Xks, lambks, Bks, Vxgs, Fks, FOCs, sls, sdrC, conds, ts), params

In [11]:
label_prior = gl.utils.class_priors(labels)

model = ssm(W, labels=labels, label_prior=label_prior)
pred, data, params = model._fit(train_ind, train_labels)
pred_labels = pred.argmax(1)

accuracy = gl.ssl.ssl_accuracy(pred_labels, labels, train_ind)
print("accuracy: %.2f%%"%accuracy)

[7.52611890e-15 1.37011353e-03 1.80570535e-03 2.13813124e-03
 2.57961917e-03 2.83619377e-03 3.70011340e-03 3.98993647e-03
 4.35796191e-03 4.73453152e-03 5.57687464e-03 5.93008887e-03]


  0%|          | 0/41 [00:00<?, ?it/s]

0 F: 222.310 foc: 25.834 sl: 38.779 slsym: 38.780 sdrC: 35.544 slCinv: 0.006 d_r+1 - dr: 0.00035


2024-04-19 18:12:37.789512: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


==== VF: -33.013 foc: 33.438 sl: 38.033 sdrC: 35.544
==== VF: -64.201 foc: 66.777 sl: 36.115 sdrC: 35.544
==== VF: -64.189 foc: 66.745 sl: 36.135 sdrC: 35.544
==== VF: -64.166 foc: 66.712 sl: 36.165 sdrC: 35.544
==== VF: -91.970 foc: 6.741 sl: 27.051 sdrC: 35.544
accuracy: 61.75%
1 F: 174.370 foc: 91.786 sl: 30.711 slsym: 30.724 sdrC: 35.544 slCinv: 0.004 d_r+1 - dr: 0.00035
==== VF: -148.450 foc: 19.947 sl: 22.921 sdrC: 35.544
==== VF: -149.270 foc: 0.100 sl: 22.364 sdrC: 35.544
==== VF: -149.270 foc: 0.003 sl: 22.361 sdrC: 35.544
==== VF: -149.270 foc: 0.000 sl: 22.361 sdrC: 35.544
==== VF: -149.270 foc: 0.000 sl: 22.361 sdrC: 35.544
accuracy: 61.31%
2 F: 158.596 foc: 26.281 sl: 29.925 slsym: 29.938 sdrC: 35.544 slCinv: 0.004 d_r+1 - dr: 0.00035
==== VF: -152.867 foc: 6.303 sl: 21.691 sdrC: 35.544
==== VF: -153.134 foc: 0.093 sl: 21.421 sdrC: 35.544
==== VF: -153.134 foc: 0.001 sl: 21.420 sdrC: 35.544
==== VF: -153.134 foc: 0.000 sl: 21.420 sdrC: 35.544
==== VF: -153.134 foc: 0.000 s

In [12]:
Xks, lambks, Bks, Vxgs, Fks, FOCs, sls, sdrC, conds, ts = data